<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#AUC" data-toc-modified-id="AUC-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>AUC</a></span><ul class="toc-item"><li><span><a href="#代码" data-toc-modified-id="代码-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>代码</a></span></li><li><span><a href="#测试" data-toc-modified-id="测试-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>测试</a></span></li></ul></li><li><span><a href="#精确度" data-toc-modified-id="精确度-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>精确度</a></span></li><li><span><a href="#排序分数-Ranking-Score" data-toc-modified-id="排序分数-Ranking-Score-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>排序分数 Ranking Score</a></span></li></ul></div>

* AUC :最常用， 整体上衡量算法的精确度


* Precision ： 只考虑排在前 L 维 的边是否预测准确


* Ranking Score ： 考虑预测边的排序



#  AUC

> AUC 指的是  ROC 曲线下的面积， 其中ROC 在信号探测中用来评价某种分类器的分类效果

AUC 可以理解为 在测试集中随机选择一条边的分数值比随机选择一条不存在的边的分数值高的概率。


* 每次随机从测试集中选择一条边，再从不存在边中随机选择一条，如果测试集中边分数值大于不存在的边的分数值， 就 +1 ， 如果两个相等， 就 +0.5， 

* 重复上述步骤 n 次， 假设 $n_{1}$ 次大于， $n_{2}$ 次分数相等


$$AUC = \frac{n_{1} + 0.5 n_{2}}{n}$$



**AUC 对样本类别是否均衡并不敏感**



**训练集选择的比例不同， AUC 值会相应的变化，一般来说， 训练集越大， AUC的 值越高**






## 代码

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
from scipy import sparse

In [17]:
def CalcAUC(MatrixAdjacency_train, MatrixAdjacency_test, Matrix_similarity, n):
    """
    计算 AUC
    
    n : 抽样比较的次数
    """

    # 只保留测试集和不存在边集合中的边的相似度（ 不保留自环）
    Matrix_similarity = np.triu(Matrix_similarity -
                                Matrix_similarity * MatrixAdjacency_train)

    Matrix_noexist = np.ones_like(
        MatrixAdjacency_train
    ) - MatrixAdjacency_train - MatrixAdjacency_test - np.eye(
        MatrixAdjacency_train.shape[0])

    # 取测试集和不存在边集合的上三角矩阵， 取出对应的相似度分值
    Test = np.triu(MatrixAdjacency_test)
    NoExist = np.triu(Matrix_noexist)

    #Test_num = np.count_nonzero(Test)
    #Non_num = np.count_nonzero(NoExist)

    Test_num = len(np.argwhere(Test == 1))
    Non_num = len(np.argwhere(NoExist == 1))

    
    # !!! 创建的随机是 正态分布， 选取其他的分布
    Test_rd = [int(x) for x in np.random.rand(1, n)[0] * Test_num]
    Non_rd = [int(x) for x in np.random.rand(1, n)[0] * Non_num]

    Test_pre = Matrix_similarity * Test
    Non_pre = Matrix_similarity * NoExist

    # test 集合存在的边的预测值

    Test_index = np.argwhere(Test == 1)
    Test_data = np.array([Test_pre[x[0], x[1]] for x in Test_index]).T

    # 不存在边的集合的边的预测值
    Non_index = np.argwhere(NoExist == 1)
    Non_data = np.array([Non_pre[x[0], x[1]] for x in Non_index]).T

    Test_rd_2 = np.array([Test_data[i] for index, i in enumerate(Test_rd)])
    Non_rd_2 = np.array([Non_data[i] for index, i in enumerate(Non_rd)])

    n1, n2 = 0, 0

    for cal_times in range(n):
        if Test_rd_2[cal_times] > Non_rd_2[cal_times]:
            n1 += 1
        elif Test_rd_2[cal_times] == Non_rd_2[cal_times]:
            n2 += 1

    auc = (n1 + 0.5 * n2) / n

    return auc

## 测试

In [3]:
def spones(array):
    """
    将矩阵中的非零元素全部替换为 1 
    """
    sparse_array = sparse.csr_matrix(array)  # 转换为稀疏矩阵
    sparse_array.data.fill(1)  # 稀疏矩阵全部替换为 1
    array = sparse_array.toarray()  # 返回密集矩阵

    return array


def DivideNet(MatrixAdajacency, ratioTrain):
    """
    划分训练集和测试集
    
    保证网络训练集连通
    """

    # 测试集的连边数目
    num_testlinks = int(
        (1 - ratioTrain) * np.count_nonzero(MatrixAdajacency) / 2)
    # 将邻接矩阵中所有的边找出来 存储到 linklist 中
    linklist = [list(i) for i in np.argwhere(np.triu(MatrixAdajacency))]
    # 为每条边都设置标志位 ， 判断是否能够删除
    MatrixAdajacency_test = np.zeros(shape=(np.shape(MatrixAdajacency)[0],
                                            np.shape(MatrixAdajacency)[1]))

    while (np.count_nonzero(MatrixAdajacency_test) < num_testlinks):

        ### 随机选择一条边
        link_index = int(np.random.rand(1) * len(linklist))
        uid1 = linklist[link_index][0]  # 边两端的节点 1
        uid2 = linklist[link_index][1]

        ### 判断所选边两端节点 uid1 和 uid2 是否可达， 若可达则放入测试集， 否则重新选边
        # 将这条边从网络中挖去，
        MatrixAdajacency[uid1, uid2] = 0
        MatrixAdajacency[uid2, uid1] = 0

        tempvector = MatrixAdajacency[uid1]  # 取出 uid1  一步可达的点 构成一维向量
        sign = 0  # 标记此边是否可以被移除，  sign = 0 表示不可， sign = 1 表示可以
        uid1TOuid2 = np.dot(tempvector,
                            MatrixAdajacency) + tempvector  # 表示 uid1 2步内可达的点

        if uid1TOuid2[0, uid2] > 0:
            sign = 1  # 两步即可到达
        else:

            count = 1
            while (len((spones(uid1TOuid2) - tempvector).nonzero()[0]) != 0):
                # 直到可达的点到达稳定状态， 仍然不能到达 uid2， 此边就不能删除
                tempvector = spones(uid1TOuid2)
                uid1TOuid2 = np.dot(
                    tempvector, MatrixAdajacency) + tempvector  # 表示 K 步 内可达的点
                count += 1
                if uid1TOuid2[0, uid2] > 0:
                    sign = 1  # 某步内可以到达
                    break

                if count >= MatrixAdajacency.shape[0]:
                    print("不可达" + str([uid1, uid2]))
                    sign = 0

        ### 如果边可以删除， 将其放入测试集中， 并从 link 集中删除
        if sign == 1:
            linklist.pop(link_index)
            MatrixAdajacency_test[uid1, uid2] = 1

        ### 如果不可以删除， 恢复原始矩阵， 也从 link 集中删除
        else:
            linklist.pop(link_index)
            MatrixAdajacency[uid1, uid2] = 1
            MatrixAdajacency[uid2, uid1] = 1

    MatrixAdajacency_Train = MatrixAdajacency
    MatrixAdajacency_Test = MatrixAdajacency_test + MatrixAdajacency_test.T

    print('    训练集计划边数：' +
          str(np.count_nonzero(MatrixAdajacency) - num_testlinks))
    print('    训练集实际边数：' + str(np.count_nonzero(MatrixAdajacency_Train)))
    print('    测试集计划边数：' + str(num_testlinks))
    print('    测试集实际边数：' + str(np.count_nonzero(MatrixAdajacency_Test)))

    return MatrixAdajacency_Train, MatrixAdajacency_Test


def check_symmetric(a, tol=1e-8):
    """
    检查是否是 对称矩阵
    """
    return np.allclose(a, a.T, atol=tol)

In [4]:
edge_lst = []
with open("lit-bm-13.txt","r") as file_txt: # 蛋白质相互作用数据集
    for i, line in enumerate(file_txt.readlines()):
        edge_tpl =  str(line.split()[0]), str(line.split()[1]),{'weight': 1}
        edge_lst.append(edge_tpl)

F = nx.from_edgelist(edge_lst)
df = nx.to_pandas_adjacency(F)
data = np.matrix(df)

In [5]:
train , test = DivideNet(data.copy(), 0.9)

    训练集计划边数：7964
    训练集实际边数：8428
    测试集计划边数：464
    测试集实际边数：866


In [6]:
def Cn(MatrixAdjacency_Train):
    """
    共同邻居算法
    """
    Matrix_similarity = np.dot(MatrixAdjacency_Train,MatrixAdjacency_Train)
    return Matrix_similarity

In [7]:
Matrix_similarity = Cn(train)

In [26]:
auc = CalcAUC(train, test, Matrix_similarity,  400000)

402
5743357
(402,)


In [27]:
auc

0.2609425

----------

# 精确度

不关注所有的预测结果， 只关注预测的前面的几条边是否预测的较为准确

> 在PPI 之中， 前面几个的预测结果较为准确，可以直接进行试验验证


**定义： 前 L 个预测的边之中， 预测准确的比例**

$$Presicion = \frac{m}{L}$$




-----

# 排序分数 Ranking Score 

考虑测试集中边在最终排序中的位置


$H = U - E^{T}$ 为未知边的集合


$r_{e}$ 表示测试边 e 在排序中的排名


测试边的排序分 ：


$$RS_{e} = \frac{r_{e}}{|H|}$$


遍历所有的边求平均值


--------

**排序的分值越小，说明预测的效果越好**